In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import openpyxl
import teradata
import plotly
import plotly.graph_objects as go
from ipywidgets import widgets
import plotly.graph_objects as go
import plotly.figure_factory as ff
import pickle
mpl.rcParams['figure.dpi']= 100

In [11]:
with open('testdata.txt', 'rb') as file:
    dflist = pickle.load(file)

In [12]:
aggoptions=['Gesamt', 'Geschlecht', 'Altersgruppe', 'Shop', 'Kalenderwoche']
for i, ddf in enumerate(dflist):
    if ddf.columns[0] in aggoptions:
        dflist[i].set_index(dflist[i].columns[0], inplace=True)
    if None in ddf.index:
        dflist[i].drop(index=[None], inplace=True)
dflist[0].rename(index={0:'Sum'}, inplace=True)

In [13]:
df1 = dflist[0].T.iloc[:7]
df2 = dflist[0].T.iloc[7:34]
df3 = dflist[0].T.iloc[34:]

catoptions=['P1', 'P2', 'P3']
aggoptions=['Gesamt', 'Geschlecht', 'Altersgruppe', 'Shop', 'Kalenderwoche']
optdict = {'P1':df1, 'P2':df2, 'P3':df3}
init = 'P1'

textbox = widgets.Dropdown(
    description='Kategorie:   ',
    value='P1',
    options=catoptions
)

aggregation = widgets.Dropdown(
    description='Aggregation:   ',
    value='Gesamt',
    options=aggoptions
)

elements = widgets.SelectMultiple(
    options=optdict[init].index,
    value=[],
    description='Elemente',
    disabled=False
)


ttrace1 = go.Table(header=dict(values=list(['index']+list(df1.columns))), cells=dict(values=[list(df1.index)]+df1.T.values.tolist()))
ttrace2 = go.Table(header=dict(values=list(['index']+list(df2.columns))), cells=dict(values=[list(df2.index)]+df2.T.values.tolist()))
ttrace3 = go.Table(header=dict(values=list(['index']+list(df3.columns))), cells=dict(values=[list(df3.index)]+df3.T.values.tolist()))

g = go.FigureWidget(data=[])
for i in range(df1.shape[0]):
    trace = go.Bar(x=df1.columns, y=df1.iloc[i], name=df1.index[i])
    g.add_trace(trace)    
g.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})

t = go.FigureWidget(data=[ttrace1])

ttlist = [ttrace1, ttrace2, ttrace3]

In [14]:
def response(change):
    
    init = change.new
    elements.options = optdict[init].index
    
    index = catoptions.index(textbox.value)
    dd = optdict[init]
    dd = (dd/dd.sum(axis=0) * 100)
    
    with g.batch_update():
        g.data = []
        for i in range(dd.shape[0]):
            trace = go.Bar(x=dd.columns, y=dd.iloc[i], name=dd.index[i])
            g.add_trace(trace)    
        g.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
    with t.batch_update():    
        t.data = []
        t.add_trace(ttlist[index])

def response2(change):
    
    cat = textbox.value
    indices = list(elements.value)
    newttrace = go.Table(header=dict(values=list(['index', df1.columns])), cells=dict(values=[indices, optdict[cat].loc[indices]]))
    
    dd = optdict[cat].loc[indices]
    dd = (dd/dd.sum(axis=0) * 100)
    
    with g.batch_update():
        g.data = []
        for i in range(dd.shape[0]):
            trace = go.Bar(x=dd.columns, y=dd.iloc[i], name=dd.index[i])
            g.add_trace(trace)    
        g.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
    with t.batch_update():    
        t.data = []
        t.add_trace(newttrace)

def response3(change):
    
    index = aggoptions.index(aggregation.value)
    
    df1 = dflist[index].T.iloc[:7]
    df2 = dflist[index].T.iloc[7:34]
    df3 = dflist[index].T.iloc[34:]
    global optdict 
    optdict = {'P1':df1, 'P2':df2, 'P3':df3}

    cat = textbox.value
    if elements.value: 
        indices = list(elements.value)
    else:
        indices = optdict[cat].index
    
    newttrace = go.Table(header=dict(values=list(['index']+list(df1.columns))), cells=dict(values=[indices]+optdict[cat].loc[indices].T.values.tolist()))
    ttrace1 = go.Table(header=dict(values=list(['index']+list(df1.columns))), cells=dict(values=[df1.index]+df1.T.values.tolist()))
    ttrace2 = go.Table(header=dict(values=list(['index']+list(df2.columns))), cells=dict(values=[df2.index]+df2.T.values.tolist()))
    ttrace3 = go.Table(header=dict(values=list(['index']+list(df3.columns))), cells=dict(values=[df3.index]+df3.T.values.tolist())) 
    global ttlist
    ttlist = [ttrace1, ttrace2, ttrace3]

    dd = optdict[textbox.value].loc[indices]
    dd = (dd/dd.sum(axis=0) * 100)
    
    with g.batch_update():
        g.data = []
        for i in range(dd.shape[0]):
            trace = go.Bar(x=dd.columns, y=dd.iloc[i], name=dd.index[i])
            g.add_trace(trace)    
        g.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
    with t.batch_update():    
        t.data = []
        t.add_trace(newttrace)


        
textbox.observe(response, names="value")
elements.observe(response2, names="value")
aggregation.observe(response3, names='value')

In [15]:
comp = widgets.HBox([textbox, elements, aggregation])
widgets.VBox([comp, g, t])